<a href="https://colab.research.google.com/github/KoganTheDev/cloud-computing-project/blob/main/cloudafterpresentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import time
import matplotlib.pyplot as plt
import io
from IPython.display import display as ipy_display
from typing import Dict, List, Callable
from firebase import firebase
from datetime import datetime
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo
from nltk.stem import PorterStemmer
local_tz = ZoneInfo("Asia/Jerusalem")
# --- Constants ---
BLUE_COLOR = '#3498db'
DARKER_BLUE_COLOR = '#2c3e50'
ORANGE_COLOR = '#FF851B'
LIGHT_BLUE_COLOR = '#f0f8ff'
LIGHTER_BLUE_COLOR = '#f7f9fc'
WHITE_COLOR = 'rgba(255,255,255,0.7)'
LIGHT_GREY_BORDER = '#ddd'
BOX_SHADOW = '0px 4px 8px rgba(0,0,0,0.2)'
BORDER_RADIUS = '12px'
PADDING = '30px'
MARGIN = '20px'

# --- Data Types ---
SensorData = Dict[str, Dict[str, float or str]]
EngineersPerformance = Dict[str, Dict[str, int]]
HistoricalSensorData = Dict[str, Dict[str, List[float] or List[str]]]


stemmer = PorterStemmer()

# --- Firebase Setup ---
FBconn = firebase.FirebaseApplication(
    'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
)
local_tz = ZoneInfo("Asia/Jerusalem")
DATABASE_PATH = 'Data'
INDOOR_SENSOR_PATH = f'{DATABASE_PATH}/indoor'
OUTDOOR_SENSOR_PATH = f'{DATABASE_PATH}/outdoor'

# --- Dummy Data (Replace with your actual data) ---
daily_tasks: List[str] = [
    "Review production logs for anomalies",
    "Check calibration of robotic arms",
    "Optimize energy consumption routines",
    "Inspect conveyor belt system",
]

engineers_performance: EngineersPerformance = {
    "Alice": {"Points": 1550, "Improvements Completed": 32},
    "Bob": {"Points": 1200, "Improvements Completed": 25},
    "Charlie": {"Points": 1800, "Improvements Completed": 40},
    "David": {"Points": 950, "Improvements Completed": 18},
}


# --- Helper Functions ---

def create_html_widget(value: str, **layout_kwargs) -> widgets.HTML:
    """Creates an HTML widget with specified layout."""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HTML(value=value, layout=layout)

def create_button(description: str, button_style: str = 'primary', **layout_kwargs) -> widgets.Button:
    """Creates a Button widget with specified layout."""
    default_layout_kwargs = {
        'width': '200px',
        'height': '60px',
        'border_radius': BORDER_RADIUS,
        'font_size': '16px'
    }
    default_layout_kwargs.update(layout_kwargs) # Override defaults
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Button(description=description, button_style=button_style, layout=layout)

def create_text_input(description: str, **layout_kwargs) -> widgets.Text:
    """Creates a Text input widget with specified layout."""
    default_layout_kwargs = {
        'width': '600px',
        'height': '40px',
        'border_radius': BORDER_RADIUS,
        'padding': '10px',
        'font_size': '16px'
    }
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Text(description=description, layout=widgets.Layout(**default_layout_kwargs))

def create_output(**layout_kwargs) -> widgets.Output:
    """Creates an Output widget with specified layout."""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.Output(layout=layout)

def create_dropdown(options: List[str], description: str, **layout_kwargs) -> widgets.Dropdown:
    """Creates a Dropdown widget."""
    default_layout_kwargs = {'width': '300px', 'margin': '10px 0'}
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.Dropdown(options=options, description=description, layout=layout)

def create_vbox(children: List[widgets.Widget], **layout_kwargs) -> widgets.VBox:
    """Creates a VBox container with specified layout."""
    default_layout_kwargs = {
        'padding': PADDING,
        'border': f'2px solid {BLUE_COLOR}',
        'border_radius': '15px',
        'margin': MARGIN,
        'background_color': LIGHT_BLUE_COLOR
    }
    default_layout_kwargs.update(layout_kwargs)
    layout = widgets.Layout(**default_layout_kwargs)
    return widgets.VBox(children=children, layout=layout)

def create_hbox(children: List[widgets.Widget], **layout_kwargs) -> widgets.HBox:
    """Creates an HBox container"""
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HBox(children = children, layout=layout)


def perform_search(query: str):
    # Stem and lowercase the input
    stemmed_query = stemmer.stem(query.lower())

    # Fetch the top 20 index from Firebase
    index_data = FBconn.get('/myTest1/Index', None)

    results = []

    if index_data and stemmed_query in index_data:
        entry = index_data[stemmed_query]

        results.append(f"<h3>Search Result for: <i>{query}</i></h3>")
        results.append(f"<b>Term:</b> {entry['term']}")
        results.append(f"<b>Count:</b> {entry['count']} occurrence(s)")

        results.append("<b>Appears in URLs:</b><ul>")
        for url in entry['DocIDs']:
            results.append(f"<li><a href='{url}' target='_blank'>{url}</a></li>")
        results.append("</ul>")
    else:
        results.append(f"<b>No matching word</b> found in the top 20 index for: <i>{query}</i>")

    return results


def fetch_latest_sensor_data() -> SensorData:
    """Fetches the latest data for indoor and outdoor sensors from Firebase."""
    indoor_data = FBconn.get(INDOOR_SENSOR_PATH, None)
    outdoor_data = FBconn.get(OUTDOOR_SENSOR_PATH, None)
    latest_indoor = {}
    if indoor_data:
        # Assuming the keys are timestamps or some ordered identifier
        # We need to find the latest data point based on the 'Time' field
        latest_key = None
        latest_time = ""
        for key, value in indoor_data.items():
            current_time = value.get('Time', '')
            if current_time > latest_time:
                latest_time = current_time
                latest_key = key
        if latest_key:
            latest_indoor = indoor_data[latest_key]

    latest_outdoor = {}
    if outdoor_data:
        latest_key = None
        latest_time = ""
        for key, value in outdoor_data.items():
            current_time = value.get('Time', '')
            if current_time > latest_time:
                latest_time = current_time
                latest_key = key
        if latest_key:
            latest_outdoor = outdoor_data[latest_key]

    return {
        "Indoor Sensor": latest_indoor,
        "Outdoor Sensor": latest_outdoor,
    }

def generate_sensor_statistics(selected_sensor: str, sensor_data: SensorData, selected_graph_attribute: str = None) -> widgets.HTML:
    """
    Generates and returns an HTML widget displaying statistics for the selected sensor.
    If selected_graph_attribute is provided, it will only display that attribute's latest value.
    """
    if selected_sensor in sensor_data and sensor_data[selected_sensor]:
        data = sensor_data[selected_sensor]
        stats_text = f"""
            <div style='display: flex; justify-content: space-between;'>
                <div style='flex: 1; margin-right: 20px;'>
        """

        found_attribute_key = None
        if selected_graph_attribute:
            # Try to find a case-insensitive match for the selected attribute in data keys
            for key in data.keys():
                if key.lower() == selected_graph_attribute.lower():
                    found_attribute_key = key
                    break

        if found_attribute_key:
            # Get the unit for the selected attribute
            unit = ""
            if selected_sensor == "Indoor Sensor":
                units_map = {"Humidity": "%", "Temperature": "°C", "Pressure": "Bar", "Distance": "Cm"}
            elif selected_sensor == "Outdoor Sensor":
                units_map = {"Humidity": "%", "Temperature": "°C", "DLIGHT": "Lux"}
            unit = units_map.get(selected_graph_attribute, "") # Use selected_graph_attribute for unit lookup

            stats_text += f"<b>{selected_graph_attribute}:</b> {data[found_attribute_key]} {unit}<br>"
            # Always include Time if it exists
            if "Time" in data:
                stats_text += f"<b>Time:</b> {data['Time']}<br>"
        else:
            # Fallback if the selected attribute (even case-insensitively) is not found
            stats_text += f"<b>No specific data available for '{selected_graph_attribute}' or attribute not found in latest sensor data.</b><br>"

        stats_text += """
                </div>
            </div>
        """
        return create_html_widget(stats_text)
    else:
        # This handles cases where no data is available for the selected sensor at all
        return create_html_widget(f"No data available for {selected_sensor}.")


def fetch_historical_sensor_data(sensor_path: str) -> HistoricalSensorData:
    """Fetches all historical data for a given sensor from Firebase."""
    data = FBconn.get(sensor_path, None)
    if data:
        # Initialize historical_data with common attributes
        historical_data: HistoricalSensorData = {"Timestamp": [], "Humidity": [], "Temperature": []}
        if sensor_path == INDOOR_SENSOR_PATH:
            historical_data.update({"Pressure": [], "Distance": []})
        elif sensor_path == OUTDOOR_SENSOR_PATH:
            historical_data.update({"DLIGHT": []}) # Ensure DLIGHT is initialized for outdoor

        for entry in data.values():
            historical_data["Timestamp"].append(entry.get("Time", ""))
            historical_data["Humidity"].append(entry.get("Humidity"))
            historical_data["Temperature"].append(entry.get("Temperature"))
            if sensor_path == INDOOR_SENSOR_PATH:
                historical_data["Pressure"].append(entry.get("Pressure"))
                historical_data["Distance"].append(entry.get("Distance"))
            elif sensor_path == OUTDOOR_SENSOR_PATH:
                historical_data["DLIGHT"].append(entry.get("DLIGHT"))
        return historical_data
    else:
        # Return empty data for all possible attributes if no data is found
        if sensor_path == INDOOR_SENSOR_PATH:
            return {"Timestamp": [], "Humidity": [], "Temperature": [], "Pressure": [], "Distance": []}
        elif sensor_path == OUTDOOR_SENSOR_PATH:
            return {"Timestamp": [], "Humidity": [], "Temperature": [], "DLIGHT": []}
        return {"Timestamp": [], "Humidity": [], "Temperature": [], "Pressure": [], "Distance": [], "DLIGHT": []}


def create_historical_plot(sensor_name: str, selected_attribute: str) -> widgets.Output:
    """Creates and returns an output widget with a single historical plot for the selected attribute."""
    if sensor_name == "Indoor Sensor":
        historical_data = fetch_historical_sensor_data(INDOOR_SENSOR_PATH)
        units = {"Humidity": "%", "Temperature": "°C", "Pressure": "Bar", "Distance": "Cm"}
        color = 'b' # Default color
    elif sensor_name == "Outdoor Sensor":
        historical_data = fetch_historical_sensor_data(OUTDOOR_SENSOR_PATH)
        units = {"Humidity": "%", "Temperature": "°C", "DLIGHT": "Lux"} # Corrected units for Outdoor
        attribute_colors = {"Humidity": 'b', "Temperature": 'r', "DLIGHT": 'orange'} # Corrected colors for Outdoor
        color = attribute_colors.get(selected_attribute, 'b') # Get specific color or default
    else:
        # Corrected handling for invalid sensor name
        error_output = create_output(layout=widgets.Layout(height='auto'))
        with error_output:
            display(widgets.HTML(f"Invalid sensor name: {sensor_name}"))
        return error_output

    if not historical_data["Timestamp"]:
        # Corrected handling for no historical data
        no_data_output = create_output(layout=widgets.Layout(height='auto'))
        with no_data_output:
            display(widgets.HTML(f"No historical data available for {sensor_name}"))
        return no_data_output

    out = widgets.Output()
    with out:
        plt.figure(figsize=(10, 6)) # Single plot, so adjust figure size
        times = historical_data["Timestamp"]
        if times and not isinstance(times[0], datetime):
            try:
                times = [datetime.strptime(t, "%H:%M:%S") if isinstance(t, str) else
                         datetime.fromtimestamp(t) if isinstance(t, (int, float)) else t
                         for t in times]
            except ValueError:
                pass

        y_data = historical_data.get(selected_attribute, [])
        unit = units.get(selected_attribute, "")

        if not y_data:
            plt.text(0.5, 0.5, f"No data for {selected_attribute}", ha='center', va='center')
            plt.title(f"{selected_attribute} over Time ({sensor_name})")
        else:
            plt.plot(times, y_data, marker='o', color=color, label=selected_attribute)
            plt.title(f"{selected_attribute} over Time ({sensor_name})")
            plt.xlabel("Time")
            plt.ylabel(f"{selected_attribute} ({unit})")
            if times and isinstance(times[0], datetime):
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
                plt.setp(plt.gca().get_xticklabels(), rotation=45, ha='right')
            valid_ydata = [y for y in y_data if y is not None]
            if valid_ydata:
                y_min, y_max = min(valid_ydata), max(valid_ydata)
                if y_min != y_max:
                    y_range = y_max - y_min
                    plt.ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
            plt.grid(True)
            plt.legend()

        plt.tight_layout()
        plt.show()
    return out


def update_sensor_data_and_plot(change) -> None:
    """Updates the statistics display and plots based on the selected sensor and attribute."""
    selected_sensor = sensor_selector.value # Get the current sensor selected
    selected_attribute = graph_selector.value # Get the current attribute selected

    latest_data = fetch_latest_sensor_data()
    # Update the display for selected sensor and graph
    # Pass selected_attribute to generate_sensor_statistics
    stats_display.value = generate_sensor_statistics(selected_sensor, latest_data, selected_attribute).value

    # Clear the output widget and display the new figure
    stats_plots.clear_output(wait=True)
    with stats_plots:
        out = create_historical_plot(selected_sensor, selected_attribute)
        display(out)

# --- Main Menu Screen Widgets ---
main_menu_title = create_html_widget(
    "<h1 style='color:#3498db; text-align:center; margin-bottom:30px;'>Robotics Lab Control Center</h1>",
    margin='0 0 20px 0'
)

# Create large, attractive menu buttons
manager_btn = create_button(
    "System Manager",
    button_style='info',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

statistics_btn = create_button(
    "Sensor Statistics",
    button_style='success',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

search_btn = create_button(
    "Search Query",
    button_style='warning',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

optimization_btn = create_button(
    "Optimization Race",
    button_style='danger',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

# Fetch the latest sensor data right before creating the dashboard_summary
# This ensures the status is up-to-date when the dashboard_summary is rendered.
latest_sensor_info = fetch_latest_sensor_data()
indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

# Create dashboard summary for main menu
dashboard_summary = create_html_widget(
    f"""
    <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
        <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
        <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                </h4>
                <p style='margin-bottom:5px; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                </p>
                <p style='margin-bottom:0; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                </p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{len(daily_tasks)}</span> tasks pending</p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>Charlie</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>1800</span> points</p>
            </div>
        </div>
    </div>
    """,
    margin='20px 0'
)

# Arrange menu buttons in a grid layout
top_row = create_hbox([manager_btn, statistics_btn], justify_content='center')
bottom_row = create_hbox([search_btn, optimization_btn], justify_content='center')
menu_buttons = create_vbox(
    [top_row, bottom_row],
    border='none',
    background_color='transparent',
    padding='10px'
)

# Welcome message with date/time
current_time = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
welcome_msg = create_html_widget(
    f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time}</p></div>"
)

# Main menu container
main_menu_container = create_vbox(
    [
        main_menu_title,
        welcome_msg,
        dashboard_summary,
        menu_buttons
    ],
    background_color=LIGHTER_BLUE_COLOR,
    border=f'2px solid {DARKER_BLUE_COLOR}',
    box_shadow=BOX_SHADOW,
    width='800px',
    margin='0 auto'
)

# --- System Manager Screen Widgets ---
manager_title = create_html_widget("<h2 style='color:#3498db;'>Robotics Lab System Manager Dashboard</h2>")
production_status_title = create_html_widget("<h3>Sensor Status</h3>")
sensor_status_display = create_html_widget("") # To display indoor/outdoor status
tasks_title = create_html_widget("<h3>Daily Optimization Tasks</h3>")
tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
leaderboard_title = create_html_widget("<h3>Engineer Optimization Race Leaderboard</h3>")
leaderboard_items = [f"<li><span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>"
                     for engineer, performance in engineers_performance.items()]
leaderboard = create_html_widget("<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>")

manager_back_btn = create_button("Back to Main Menu", button_style='info')
manager_output = create_output()
manager_container = create_vbox(
    [
        manager_title,
        production_status_title,
        widgets.VBox([sensor_status_display], layout=widgets.Layout(border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', padding='15px', margin_bottom='15px',
                                                                     background_color=WHITE_COLOR)),
        tasks_title,
        tasks_list,
        leaderboard_title,
        leaderboard,
        manager_back_btn,
        manager_output
    ]
)

def update_sensor_status() -> None:
    """Updates the sensor status display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    indoor_status = "Available" if latest_data.get("Indoor Sensor") else "Not Available"
    outdoor_status = "Available" if latest_data.get("Outdoor Sensor") else "Not Available"
    status_text = f"""
        <b>Indoor Sensor:</b> <span style='font-weight:bold;'>{indoor_status}</span><br>
        <b>Outdoor Sensor:</b> <span style='font-weight:bold;'>{outdoor_status}</span>
    """
    sensor_status_display.value = status_text

# --- Search Query Screen Widgets ---
query_title = create_html_widget("<h2 style='color:#3498db;'>Search Query</h2>")
query_input = widgets.Text(
    description="Enter your query:",
    style={'description_width': 'initial'},  # This prevents truncation
    layout=widgets.Layout(
        width='600px',
        height='40px',
        border_radius=BORDER_RADIUS,
        padding='10px',
        font_size='16px'
    )
)
search_button = create_button("Search")
query_results = create_output(layout=widgets.Layout(height='300px', border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', overflow='auto', background_color='rgba(255,255,255,0.8)', padding='15px'))
query_back_btn = create_button("Back to Main Menu", button_style='info')
query_container = create_vbox(
    [
        query_title,
        query_input,
        search_button,
        query_results,
        query_back_btn
    ]
)

# --- Statistics Screen Widgets ---
stats_title = create_html_widget("<h2 style='color:#2c3e50; text-align:center;'>Sensor Statistics</h2>")
stats_display = create_html_widget(
    "",
    border_radius='8px',
    padding='15px',
    margin_bottom='15px',
    background_color='rgba(255,255,255,0.9)',
    box_shadow=BOX_SHADOW
)


sensor_selector = create_dropdown(
    options=["Indoor Sensor", "Outdoor Sensor"],
    description="<b style='color:#2c3e50;'>Sel_Sensor:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'}, # Added this line
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)

graph_selector = create_dropdown(
    options=["Humidity", "Temperature", "Pressure", "Distance"], # Default options, will be updated
    description="<b style='color:#2c3e50;'>Sel_Graph:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'}, # Added this line
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)


# NEW WIDGETS: To display selected sensor and graph
selected_info_display = create_html_widget(
    "",
    border_radius='8px',
    padding='10px',
    margin_bottom='10px',
    background_color='rgba(240,248,255,0.8)', # Lighter background for these
    box_shadow=BOX_SHADOW
)

stats_back_btn = create_button("Back to Main Menu", button_style='info', layout=widgets.Layout(width='200px', height='50px', border_radius='10px', font_size='16px', margin='20px auto'))

# Create a dedicated output widget for plots with proper layout
stats_plots = widgets.Output(layout=widgets.Layout(
    min_height='600px',
    width='100%',
    border=f'1px solid {LIGHT_GREY_BORDER}',
    border_radius='8px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW,
    padding='20px',
    overflow='visible'
))

stats_container = create_vbox(
    [
        stats_title,
        sensor_selector,
        graph_selector,
        selected_info_display, # NEW: Added here
        stats_display,
        stats_plots,
        stats_back_btn
    ],
    layout=widgets.Layout(padding='30px', border=f'2px solid {DARKER_BLUE_COLOR}', border_radius='15px', margin='20px auto', background_color=LIGHTER_BLUE_COLOR, box_shadow=BOX_SHADOW)
)

# --- Optimization Race Screen Widgets ---
optimization_title = create_html_widget("<h2 style='color:#3498db;'>Optimization Race</h2>")
optimization_leaderboard_title = create_html_widget("<h3>Current Standings</h3>")
optimization_leaderboard = create_html_widget("")
optimization_tasks_title = create_html_widget("<h3>Your Daily Tasks</h3>")
optimization_tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
optimization_back_btn = create_button("Back to Main Menu", button_style='info')
optimization_output = create_output()
optimization_container = create_vbox(
    [
        optimization_title,
        optimization_leaderboard_title,
        optimization_leaderboard,
        optimization_tasks_title,
        optimization_tasks_list,
        optimization_back_btn,
        optimization_output
    ]
)

def update_optimization_leaderboard() -> None:
    """Updates the leaderboard displayed on the optimization screen."""
    sorted_engineers = sorted(engineers_performance.items(), key=lambda item: item[1]['Points'], reverse=True)
    leaderboard_items = []
    medal_icons = {
        0: "🥇",
        1: "🥈",
        2: "🥉"
    }
    for i, (engineer, performance) in enumerate(sorted_engineers):
        medal = medal_icons.get(i, "")
        leaderboard_items.append(f"<li>{medal} <span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>")
    optimization_leaderboard.value = "<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>"

#--- ScreenManagement Functions ---

current_screen = "main_menu"
displayed_screen = widgets.VBox([])

# ... (your existing screen management functions) ...

def show_main_menu() -> None:
    """Displays the main menu screen and updates the current time."""
    global current_screen, displayed_screen
    current_screen = "main_menu"
    clear_output(wait=True)

    # Update the welcome message with the current time right before displaying
    current_time_str = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
    welcome_msg.value = f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time_str}</p></div>"

    # Re-fetch and update sensor status for the dashboard summary when returning to main menu
    global dashboard_summary # Ensure dashboard_summary is accessible
    latest_sensor_info = fetch_latest_sensor_data()
    indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
    outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

    # Re-create dashboard_summary to reflect the new sensor status
    # This might be less efficient if dashboard_summary is very complex,
    # but for HTML widgets, it's generally fine.
    dashboard_summary.value = f"""
        <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
            <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
            <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                    </h4>
                    <p style='margin-bottom:5px; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                    </p>
                    <p style='margin-bottom:0; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                    </p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                    </h4>
                    <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{len(daily_tasks)}</span> tasks pending</p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                    </h4>
                    <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>Charlie</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>1800</span> points</p>
                </div>
            </div>
        </div>
        """

    displayed_screen.children = [main_menu_container]
    display(displayed_screen)

# ... (rest of your code) ...

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    displayed_screen.children = [manager_container]
    display(displayed_screen)
    update_sensor_status()

def show_query_screen() -> None:
    """Displays the query screen."""
    global current_screen, displayed_screen
    current_screen = "query"
    clear_output(wait=True)
    displayed_screen.children = [query_container]
    display(displayed_screen)

def show_statistics_screen() -> None:
    """Displays the statistics screen."""
    global current_screen, displayed_screen
    current_screen = "statistics"
    clear_output(wait=True)
    displayed_screen.children = [stats_container]
    display(displayed_screen)

    # Initialize graph selector options based on the default sensor
    if sensor_selector.value == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif sensor_selector.value == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLIGHT"]

    # Trigger an initial update for both statistics and plot
    update_sensor_data_and_plot(None) # Call with None as change object is not relevant for initial load

def show_optimization_screen() -> None:
    """Displays the optimization screen."""
    global current_screen, displayed_screen
    current_screen = "optimization"
    clear_output(wait=True)
    displayed_screen.children = [optimization_container]
    display(displayed_screen)
    update_optimization_leaderboard()

def update_screen() -> None:
    """Updates the displayed screen based on the current screen variable."""
    if current_screen == "main_menu":
        show_main_menu()
    elif current_screen == "manager":
        show_manager_screen()
    elif current_screen == "query":
        show_query_screen()
    elif current_screen == "statistics":
        show_statistics_screen()
    elif current_screen == "optimization":
        show_optimization_screen()

# --- Event Handlers ---

def handle_main_menu_navigation(button: widgets.Button) -> None:
    """Handles navigation from the main menu screen."""
    if button.description == "System Manager":
        show_manager_screen()
    elif button.description == "Sensor Statistics":
        show_statistics_screen()
    elif button.description == "Search Query":
        show_query_screen()
        query_input.value = ""  # Clear the text input box
        query_results.clear_output()  # Clear previous search results

    elif button.description == "Optimization Race":
        show_optimization_screen()

def handle_back_to_menu(button: widgets.Button) -> None:
    """Handles back to main menu navigation from any screen."""
    show_main_menu()

def handle_search(button: widgets.Button) -> None:
    """Handles the search query and displays results."""
    query = query_input.value
    results = perform_search(query)
    query_results.clear_output(wait=True)
    with query_results:
        display(widgets.HTML("<br>".join(results)))

def on_sensor_selector_change(change) -> None:
    """Updates graph options and then the plot based on the selected sensor."""
    selected_sensor = change['new']
    if selected_sensor == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif selected_sensor == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLIGHT"]
    # Ensure the graph selector has a valid option selected after options change
    # IMPORTANT: Keep this check to prevent errors if the new sensor doesn't have the previously selected attribute
    if graph_selector.value not in graph_selector.options:
        graph_selector.value = graph_selector.options[0] if graph_selector.options else None

    # Trigger an update for both statistics and plot
    update_sensor_data_and_plot(None) # Call with None as change object is not relevant here


# --- Widget Event Assignments ---
# Main menu navigation
manager_btn.on_click(handle_main_menu_navigation)
statistics_btn.on_click(handle_main_menu_navigation)
search_btn.on_click(handle_main_menu_navigation)
optimization_btn.on_click(handle_main_menu_navigation)

# Back buttons
manager_back_btn.on_click(handle_back_to_menu)
stats_back_btn.on_click(handle_back_to_menu)
query_back_btn.on_click(handle_back_to_menu)
optimization_back_btn.on_click(handle_back_to_menu)

# Search functionality
search_button.on_click(handle_search)

# Sensor selector in statistics screen
sensor_selector.observe(on_sensor_selector_change, names='value')

# Graph selector in statistics screen
graph_selector.observe(update_sensor_data_and_plot, names='value')


# --- Initial Display ---
show_main_menu()

In [84]:
# %%
!pip install paho-mqtt
!pip install firebase
import time
import json
import paho.mqtt.client as mqtt
from datetime import datetime
import pytz
from firebase import firebase


class MqttConnection:
    _instance = None  # Singleton instance

    @classmethod
    def get_instance(cls, DBLink=None):
        if cls._instance is None and DBLink is not None:
            cls._instance = cls(DBLink)
        return cls._instance

    def __init__(self, DBLink):
        if MqttConnection._instance is not None:
            raise Exception("MqttConnection is a singleton.")

        self.DBLink = DBLink
        self.FBconn = firebase.FirebaseApplication(DBLink, None)
        self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
        self.connected = False

    def insert_to_db(self, path, data):
        try:
            # Read existing data at this path
            existing = self.FBconn.get('/', f'Data/{path}')
            if existing is None:
                existing = {}

            # Merge existing data with new data (new data overwrites existing keys)
            merged_data = {**existing, **data}

            self.FBconn.put('/', f'Data/{path}', merged_data)
            print(f"Data saved to Firebase at path: Data/{path}")
        except Exception as e:
            print(f"Failed to insert data to Firebase: {e}")



    def on_connect(self, client, userdata, flags, rc, properties=None):
        if rc == 0:
            self.connected = True
            print("✅ Connected to MQTT Broker. Subscribing to topics...")
            client.subscribe("braude/D106/indoor")
            client.subscribe("braude/D106/outdoor")
            print("✅ Successfully subscribed to topics.")
        else:
            print(f"❌ Failed to connect, return code {rc}")
            self.connected = False

    def on_disconnect(self, client, userdata, rc, properties=None):
        self.connected = False
        if rc != 0:
            for i in range(5):
                wait_time = (i + 1) * 2
                print(f"🔄 Disconnected. Reconnecting in {wait_time}s (attempt {i+1}/5)...")
                time.sleep(wait_time)
                try:
                    client.reconnect()
                    print("✅ Reconnection attempt made.")
                    break
                except Exception as e:
                    print(f"❌ Reconnection failed: {e}")

    # Moved inside the class
    def on_message(self, client, userdata, msg):
        topic = msg.topic
        payload = msg.payload.decode('utf-8')
        try:
            sensor_data = json.loads(payload)
            timestamp = int(time.time())

            local_tz = pytz.timezone("Israel")
            sensor_data["Time"] = datetime.now(local_tz).strftime("%H:%M:%S")
            print(sensor_data)
            if topic == "braude/D106/indoor":
                # Now self refers to the MqttConnection instance
                self.insert_to_db(f"indoor/{timestamp}", sensor_data)
            elif topic == "braude/D106/outdoor":
                # Now self refers to the MqttConnection instance
                self.insert_to_db(f"outdoor/{timestamp}", sensor_data)
        except json.JSONDecodeError as e:
            print(f"❌ JSON decode error: {e}\nPayload: {payload}")

    # Moved inside the class
    def mqtt_handler(self):
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_message = self.on_message

        try:
            print("🔌 Attempting to connect to MQTT broker...")
            conn_result = self.client.connect("test.mosquitto.org", 1883, keepalive=60)
            print(f"Connection result code: {conn_result}")
            self.client.loop_start()

            timeout = time.time() + 10
            while time.time() < timeout and not self.connected:
                time.sleep(0.2)

            return self.connected

        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False


# --- Main execution ---
if __name__ == "__main__":
    DBLink = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"

    print("🚀 Initializing MQTT connection...")
    global_mqtt_connection = MqttConnection.get_instance(DBLink)

    if not global_mqtt_connection.connected:
        success = global_mqtt_connection.mqtt_handler()
        print(f"🔍 MQTT connection result: {'Connected' if success else 'Failed'}")

    # Let it run for a while to receive messages
    try:
        time.sleep(30)
    finally:
        global_mqtt_connection.client.loop_stop()
        print("🛑 MQTT loop stopped.")

🚀 Initializing MQTT connection...
🔌 Attempting to connect to MQTT broker...
Connection result code: 0
✅ Connected to MQTT Broker. Subscribing to topics...
✅ Successfully subscribed to topics.
🔍 MQTT connection result: Connected
{'Temperature': 22.4, 'Time': '18:20:02'}
Data saved to Firebase at path: Data/indoor/1747840802
{'Distance': 682.762, 'Time': '18:20:03'}
Data saved to Firebase at path: Data/indoor/1747840803
{'Temperature': 22.39, 'Time': '18:20:04'}
Data saved to Firebase at path: Data/indoor/1747840804
{'Distance': 691.524, 'Time': '18:20:05'}
Data saved to Firebase at path: Data/indoor/1747840805
{'Temperature': 22.4, 'Time': '18:20:05'}
Data saved to Firebase at path: Data/indoor/1747840805
{'Temperature': 22.38, 'Time': '18:20:06'}
Data saved to Firebase at path: Data/indoor/1747840806
{'Distance': 687.48, 'Time': '18:20:06'}
Data saved to Firebase at path: Data/indoor/1747840806
{'Distance': 687.48, 'Time': '18:20:06'}
Data saved to Firebase at path: Data/indoor/1747840